In [115]:
# Import libraries
import matplotlib.pyplot as plt
import random

In [116]:
# Node Class
class Node ():
    def __init__(self, parent = None, position = None, direction = None):
        self.parent = parent
        self.position = position
        self.direction = direction
        
        self.g = 0
        self.h = 0
        self.f = 0
        self.t = 0 # turning cost
    
    def __eq__(self, other):
        return self.position == other.position

In [117]:
# calculate turning cost
# note: down: x positive, right: y positive 
# global variable

iteration_count = 0

In [118]:
# A Star Algorithm
# Assumption:
# Maximum speed value: 2m/s
# Size grid element: 2m
# Maximum rotation speed: pi/2 (or 90 degrees)
# Starting position rotation: 0
# AGVs will not turn 180 degree

# Improves A* algorithm: use movement keyword to search faster 

def astar(maze, start, end, turning): # 0: no turning cost, 1: turning cost is considered
    STEP_COST = 1
    
    start_node = Node(None, start, (-1, 0)) #The AGV is facing down 
    start_node.g = start_node.f = start_node.h = start_node.t = 0
    
    end_node = Node(None, end) 
    end_node.g = end_node.h = end_node.f = end_node.t= 0
    
    open_list = []
    closed_list = []
    
    open_list.append(start_node)
    
    # start the searching loop
    while len(open_list) > 0:
        # get the current node (with the least f score)
        current_node = open_list[0]
        current_index = 0
        
        for index, node in enumerate(open_list):
            if node.f < current_node.f:
                current_node = node
                current_index = index
        
        # pop the current node out of the open list and add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)
        
        # check if the current node is the goal
        if current_node == end_node:
            path = [] # contain the path from start to end node
            
            while current_node is not None:
                
                path.append(current_node.position)
                current_node = current_node.parent
                
            return path[::-1]
        
        # Generate neighbor nodes from current node
        neighbors = []
    
        
        for new_position in [(0, 1),(0, -1),(1, 0),(-1, 0)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            
            # check if the new node is out of bound
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue
            
            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue
            
            # Create new node
            new_node = Node(current_node, node_position, new_position) # New position is add for the current direction of that node

            neighbors.append(new_node)
            

        # Loop through children
        for neighbor in neighbors:
            if neighbor in closed_list:
                continue
            
            # Create the f, g, t and h values
            
            # Calculate turning cost 
            if (current_node.direction == neighbor.direction):
                neighbor.t = 0
            else: 
                neighbor.t = 1
                
            neighbor.g = current_node.g + STEP_COST
            neighbor.h = ((neighbor.position[0] - end_node.position[0])**2 + (neighbor.position[1] - end_node.position[1])**2)**0.5
            if (turning == 0):
                neighbor.f = neighbor.g + neighbor.h 
            else:
                neighbor.f = neighbor.g + neighbor.h + neighbor.t
                    
            # Skip if the neighbor is in the open list with a higher or equal f value
            if any(
                neighbor.position == open_node.position and neighbor.f >= open_node.f
                for open_node in open_list
            ):
                continue
                
            open_list.append(neighbor)
            
    return None # if no path is found

In [119]:
def turns_counter(path):
    if len(path) < 3:
        return 0  # Not enough points to form a turn

    turns = 0

    for i in range(1, len(path) - 1):
        # Get the current point and the previous and next points
        p_prev = path[i - 1]
        p_curr = path[i]
        p_next = path[i + 1]

        # Calculate directions
        direction1 = (p_curr[0] - p_prev[0], p_curr[1] - p_prev[1])  # Direction from previous to current
        direction2 = (p_next[0] - p_curr[0], p_next[1] - p_curr[1])  # Direction from current to next

        # Check if the direction has changed
        if direction1 != direction2:
            turns += 1

    return turns 

In [120]:
def generate_random(map, times): #times: how many random start-end points to generate?
    points = []
    count = 0
    # random start and end point
    while count < times:
        start_x = random.randint(0, len(map) - 1)  
        start_y = random.randint(0, len(map) - 1) 

        end_x = random.randint(0, len(map) - 1) 
        end_y = random.randint(0, len(map) - 1) 

        if not (start_x == end_x and start_y == end_y):
            start_point = (start_x, start_y)
            end_point = (end_x, end_y)
            
            path_no_turn = astar(map, start_point, end_point, 0)
            path_turn = astar(map, start_point, end_point, 1)
            
            print(f'({start_x:<2},{start_y:<2})     ({end_x:<2},{end_y:<2})      {len(path_no_turn):<5}     {turns_counter(path_no_turn):<5}       {turns_counter(path_turn)}')
            count += 1
    
    return points

In [130]:
# Testing map 
# Size 30x30 with no obstacle for testing 

map = [
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],   
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],   
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],   
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],   
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],   
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],   
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
]

print(f'Start       End          Length    No turn     turn')
print("--------------------------------------------------------")
generate_random(map, 1000)


Start       End          Length    No turn     turn
--------------------------------------------------------
(15,19)     (0 ,11)      24        16          4
(28,29)     (4 ,11)      43        36          9
(21,15)     (17,21)      11        7           2
(17,25)     (9 ,11)      23        15          5
(23,3 )     (25,9 )      9         3           1
(8 ,10)     (15,29)      27        13          4
(12,27)     (27,9 )      34        29          8
(14,8 )     (3 ,0 )      20        16          4
(4 ,11)     (25,4 )      29        14          4
(26,0 )     (20,11)      18        11          4
(5 ,22)     (13,28)      15        12          3
(1 ,12)     (29,13)      30        2           1
(27,20)     (12,9 )      27        22          6
(18,24)     (4 ,1 )      38        27          8
(0 ,14)     (7 ,23)      17        13          4
(18,0 )     (4 ,2 )      17        4           1
(9 ,22)     (13,10)      17        7           2
(7 ,5 )     (19,24)      32        23          6
(7 ,2 )  

[]